In [4]:
import pandas as pd

In [5]:
import re

# Data

In [6]:
words = pd.read_csv('data/wikidata_results.tsv','\t')

/tmp/ipykernel_646286/2847847135.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  words = pd.read_csv('data/wikidata_results.tsv','\t')


In [7]:
words

,entity,entityLabel,entity_male_label,entity_female_label
0,http://www.wikidata.org/entity/Q76477,neveu ou nièce,neveu,nièce
1,http://www.wikidata.org/entity/Q76666,cousin germain,cousin germain,cousine germaine
2,http://www.wikidata.org/entity/Q80687,secrétaire,secrétaire,secrétaire
3,http://www.wikidata.org/entity/Q81096,ingénieur ou ingénieure,ingénieur,ingénieure
4,http://www.wikidata.org/entity/Q82594,informaticien ou informaticienne,informaticien,informaticienne
...,...,...,...,...
52148,http://www.wikidata.org/entity/Q45199,marin,NaN,marine
52149,http://www.wikidata.org/entity/Q45713,Titan,NaN,titanide
52150,http://www.wikidata.org/entity/Q54128,domestique,NaN,domestique
52151,http://www.wikidata.org/entity/Q59163,pilier,NaN,pilière


In [8]:
where = words.entity_male_label.isna()
words.entity_male_label[where] = words.entityLabel[where]

In [9]:
from collections import Counter

In [10]:
masculine2feminine = {}
for _, word in words.iterrows():
    masculine = word.entity_male_label.lower().strip()
    masculine2feminine.setdefault(masculine, Counter())
    feminine = word.entity_female_label.lower().strip()
    masculine2feminine[masculine][feminine]+=1

In [11]:
for m, fs in masculine2feminine.items():
    masculine2feminine[m] = fs.most_common(1)[0][0]     
    

def desc(masculine2feminine):
    sorted_m2f = {}
    for m in sorted(masculine2feminine, key=len, reverse=True):
        sorted_m2f[m] = masculine2feminine[m]
    return sorted_m2f

masculine2feminine = desc(masculine2feminine)

masculine2feminine

## Analysis

In [116]:
m_len,f_len=[],[]#set(),set()
for m, f in masculine2feminine.items():
    l = len(model(m))
   # if l not in m_len:
      #  print(l, m,f)
      #  m_len.add(l)
    m_len.append(len(model(m)))
    f_len.append(len(model(f)))

In [117]:
import seaborn as sns

In [118]:
pd.DataFrame(m_len).describe()

,0
count,48648.000000
mean,4.319191
std,1.343991
min,1.000000
25%,4.000000
50%,4.000000
75%,4.000000
max,24.000000


In [122]:
fig=sns.displot(m_len,discrete=True)

In [123]:
fig.savefig("dis_m_len.png")

# Translation

def translate_replace(text):
    text = text.strip().lower()
    text = re.sub(r'\s+',' ',text)
    for m, f in masculine2feminine.items():
        m = re.sub(r"[\(\)]","",m).strip()
        old_text = text
        text = re.sub(fr"\b{m}\b",f,text)#text.replace(m,f)#
        if text!= old_text:
            print(m,f)
    return text

In [1]:
from pathlib import Path

In [2]:
import spacy

/home/lerner/anaconda3/envs/lightning/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = spacy.load("fr_dep_news_trf")

In [4]:
standard_root = Path('data/corpus-francais-inclusif/vfs/')

In [71]:
for path in standard_root.glob('*.txt'):
    print(path)
    with open(path, 'rt') as file:
        text = file.read()
    print(text)
    print("\t",translate(text))
    print("\n***************\n")

data/corpus-francais-inclusif/vfs/coord-1974-giscard.txt
Martiniquaises, Martiniquais, me voici donc à la Martinique où personne n'est jamais venu sans aimer à la fois cette terre et ce peuple. Voici donc les Martiniquaises et les Martiniquais rassemblés pour accueillir le President de la République qu'ils ont élu eux-mêmes démocratiquement.

https://www.vie-publique.fr/discours/262820-allocution-de-m-valery-giscard-destaing-place-de-la-savane-fort-de-f

	 martiniquaises , martiniquais , me voici donc à la martinique où personne n' est jamais venu sans aimer à la fois cette terre et ce peuple . voici donc les martiniquaises et les martiniquais rassemblés pour accueillir le president de la république qu' ils ont élu eux - mêmes démocratiquement . 

 


***************

data/corpus-francais-inclusif/vfs/coord-1981-giscard.txt
Mes chers amis, ceux qui sont à l'intérieur, ceux qui sont au dehors, car je m'excuse auprès d'eux, le chapiteau était trop petit, mais j'espère que la sonorisation


with open("data/corpus-francais-inclusif/vfs/coord-1981-giscard.txt", 'rt') as file:
    text = file.read()
print(text)
    

In [12]:
text = """Merci à tous d'être nombreux aujourd'hui à Paris pour réaffirmer que le travail doit venir avant la Bourse.

Merci, bien entendu, aux militants Force Ouvrière, aux non-syndiqués et aux adhérents d'autres organisations qui, en étant présent aujourd'hui, montrent que leur objectif, c'est d'abord et avant tout la défense et la représentation, en toute indépendance, des intérêts des travailleurs actifs, chômeurs et retraités.

Si nombreux sont les salariés qui souhaitent s'arrêter, c'est à la fois qu'ils espèrent que leurs enfants trouveront plus facilement un travail mais aussi parce que, tant dans le public que dans le privé, les conditions de travail se détériorent, le stresse augmente, c'est-à-dire l'exploitation et l'aliénation.

Par définition, il sera toujours à côté de ceux qui souffrent, de ceux qui se font exploiter, et de ceux qui veulent que ça change pour eux et leurs enfants."""

In [13]:
tokens = model(text)

/home/lerner/anaconda3/envs/lightning/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [14]:
def is_masc(token):
    gender = token.morph.get("Gender")
    return gender and gender[0] == "Masc" 

In [16]:
def is_plur(token):    
    num = token.morph.get("Number")
    return num and num[0] == "Plur"

In [21]:
def translate(text):
    translation = []
    tokens = model(text)
    todo = set()
    for token in tokens:
       # if token.like_url:
        #    continue
        if not is_masc(token):            
            translation.append(token.lower_)
            continue
        lemma = token.lemma_
        if lemma in masculine2feminine:
            print(token, end = " : ")
            for child in token.children:
                print(f"{child} ({list(child.children)})",end = " ")
                todo.add(child.i)
            f = masculine2feminine[lemma]
            print()
            if is_plur(token):
                f+="s"
            translation.append(f"{token.lower_} et {f}")
        else:
            translation.append(token.lower_)
    for i in todo:
        token = tokens[i]
        if is_masc(token):
            if is_plur(token):
                translation[i] = translation[i][:-1]+".es"
            else:
                translation[i]+=".e"
    return " ".join(translation)

In [22]:
print(translate(text))

militants : bien ([entendu]) , ([]) aux ([]) Force ([Ouvrière]) non-syndiqués ([,, aux]) adhérents ([et, aux, organisations]) 
travailleurs : des ([]) actifs ([]) chômeurs ([,]) retraités ([et]) 
chômeurs : , ([]) 
retraités : et ([]) 
salariés : nombreux ([Si]) sont ([]) les ([]) souhaitent ([qui, arrêter]) 
merci à tous d' être nombreux aujourd'hui à paris pour réaffirmer que le travail doit venir avant la bourse . 

 merci , bien entendu , aux militants et militantes force ouvrière , aux non-syndiqué.es et aux adhérent.es d' autres organisations qui , en étant présent aujourd'hui , montrent que leur objectif , c' est d' abord et avant tout la défense et la représentation , en toute indépendance , des intérêts des travailleurs et travailleuses actif.es , chômeurs et demandeuse d'emploi.es et retraités et retraitée.es . 

 si nombreu.es sont les salariés et salariées qui souhaitent s' arrêter , c' est à la fois qu' ils espèrent que leurs enfants trouveront plus facilement un travail m

## viz

In [ ]:
# from spacy import displacy
displacy.serve(model(text), style="dep")